## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [2]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [4]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [7]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result



In [9]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [10]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
5,ambitious olsson wins triple jump,5
6,antic delighted with record breaking barca,6
7,aussie qualifier stosur wastes four memphis match,7
8,aust addresses un security council over iraq,8
9,australia is locked into war timetable opp,9


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes (it take 6 minutes on my laptop)

In [11]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [12]:
'''
Preview 'processed_docs'
'''
processed_docs[4309:4311]

4309    [rain, hamper, goat, product]
4310    [rain, help, dampen, bushfir]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [13]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [14]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [15]:
# Doku Befehle fürs Dictionary: https://radimrehurek.com/gensim/corpora/dictionary.html
print(dictionary.token2id)
# Anzahl: print(dictionary.dfs) 

{'broadcast': 0, 'communiti': 1, 'decid': 2, 'licenc': 3, 'awar': 4, 'defam': 5, 'wit': 6, 'call': 7, 'infrastructur': 8, 'protect': 9, 'summit': 10, 'aust': 11, 'rise': 12, 'staff': 13, 'strike': 14, 'affect': 15, 'australian': 16, 'travel': 17, 'ambiti': 18, 'jump': 19, 'olsson': 20, 'tripl': 21, 'win': 22, 'antic': 23, 'barca': 24, 'break': 25, 'delight': 26, 'record': 27, 'aussi': 28, 'match': 29, 'memphi': 30, 'qualifi': 31, 'stosur': 32, 'wast': 33, 'address': 34, 'council': 35, 'iraq': 36, 'secur': 37, 'australia': 38, 'lock': 39, 'timet': 40, 'contribut': 41, 'million': 42, 'birthday': 43, 'celebr': 44, 'robson': 45, 'ahead': 46, 'bathhous': 47, 'plan': 48, 'championship': 49, 'cycl': 50, 'hop': 51, 'launceston': 52, 'boost': 53, 'paroo': 54, 'suppli': 55, 'water': 56, 'bill': 57, 'blizzard': 58, 'buri': 59, 'state': 60, 'unit': 61, 'brigadi': 62, 'dismiss': 63, 'harass': 64, 'report': 65, 'troop': 66, 'arriv': 67, 'british': 68, 'combat': 69, 'daili': 70, 'kuwait': 71, 'bryant

** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [16]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary_filtered = dictionary.filter_extremes(no_below=15, no_above=0.1)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [17]:
dictionary

In [18]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
# TODO
bow_corpus = []
for sentence in processed_docs:
    bow_corpus.append(dictionary.doc2bow(sentence))
    
# Musterlösung:
# bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [19]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [20]:
bow_corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1)],
 [(11, 1), (12, 1), (13, 1), (14, 1)],
 [(14, 1), (15, 1), (16, 1), (17, 1)],
 [(18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)],
 [(11, 1), (32, 1), (33, 1), (34, 1), (35, 1)],
 [(36, 1), (37, 1), (38, 1)],
 [(34, 1), (36, 1), (39, 1), (40, 1)],
 [(22, 1), (25, 1), (41, 1), (42, 1), (43, 1)],
 [(44, 1), (45, 1)],
 [(46, 1), (47, 1), (48, 1), (49, 1)],
 [(45, 1), (50, 1), (51, 1), (52, 1)],
 [(53, 1), (54, 1), (55, 1), (56, 1)],
 [(57, 1), (58, 1), (59, 1), (60, 1)],
 [(60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)],
 [(66, 1), (67, 1), (68, 1), (69, 1), (70, 1)],
 [(71, 1), (72, 1), (73, 1), (74, 1)],
 [(75, 1), (76, 1), (77, 1), (78, 1)],
 [(79, 1), (80, 1), (81, 1), (82, 1)],
 [(44, 1), (83, 1), (84, 1), (85, 1)],
 [(86, 1), (87, 1), (88, 1), (89, 1), (90, 1)],
 [(91, 1), (92, 1), (93, 1)],
 [(94, 1

In [21]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [22]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

# TODO
tfidf = models.TfidfModel(bow_corpus) # fit model

In [23]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tfidf[bow_corpus] # apply model to corpus

In [24]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.50554610985785686)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [25]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = models.LdaMulticore(bow_corpus, id2word=dictionary, num_topics=10, passes = 50)

In [26]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.025*"closer" + 0.021*"test" + 0.018*"lead" + 0.016*"take" + 0.014*"south" + 0.012*"law" + 0.012*"author" + 0.012*"inquiri" + 0.012*"crew" + 0.011*"bushfir" 
Words: 0


Topic: 0.054*"polic" + 0.024*"help" + 0.023*"miss" + 0.023*"seek" + 0.021*"urg" + 0.016*"search" + 0.012*"region" + 0.012*"arrest" + 0.011*"train" + 0.010*"offic" 
Words: 1


Topic: 0.025*"aust" + 0.021*"drought" + 0.019*"australia" + 0.017*"record" + 0.016*"break" + 0.015*"troop" + 0.015*"look" + 0.014*"rain" + 0.012*"dead" + 0.012*"timor" 
Words: 2


Topic: 0.060*"water" + 0.033*"warn" + 0.018*"worker" + 0.017*"rise" + 0.015*"industri" + 0.013*"rudd" + 0.012*"busi" + 0.012*"union" + 0.011*"begin" + 0.011*"continu" 
Words: 3


Topic: 0.019*"push" + 0.018*"elect" + 0.016*"labor" + 0.016*"howard" + 0.015*"high" + 0.014*"say" + 0.013*"market" + 0.013*"concern" + 0.012*"chief" + 0.011*"leader" 
Words: 4


Topic: 0.033*"charg" + 0.031*"crash" + 0.029*"polic" + 0.029*"kill" + 0.029*"court" + 0.027*"face" + 0.020*"att

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [31]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
# TODO
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=10, passes = 2)

In [32]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.012*"charg" + 0.012*"court" + 0.011*"teen" + 0.009*"face" + 0.009*"stand" + 0.008*"blue" + 0.008*"control" + 0.008*"polic" + 0.008*"market" + 0.007*"high"


Topic: 1 Word: 0.025*"crash" + 0.024*"polic" + 0.020*"investig" + 0.018*"kill" + 0.016*"miss" + 0.014*"search" + 0.010*"dead" + 0.010*"fatal" + 0.009*"bodi" + 0.009*"bomb"


Topic: 2 Word: 0.013*"stab" + 0.012*"guilti" + 0.012*"drought" + 0.012*"murray" + 0.010*"black" + 0.008*"plead" + 0.008*"canberra" + 0.007*"grant" + 0.007*"charg" + 0.007*"thousand"


Topic: 3 Word: 0.011*"troop" + 0.011*"crew" + 0.010*"iraq" + 0.009*"qanta" + 0.009*"howard" + 0.009*"solomon" + 0.008*"takeov" + 0.008*"aust" + 0.007*"approv" + 0.007*"sailor"


Topic: 4 Word: 0.013*"coast" + 0.011*"south" + 0.010*"north" + 0.010*"west" + 0.009*"timor" + 0.009*"storm" + 0.009*"gold" + 0.009*"condit" + 0.008*"east" + 0.008*"iemma"


Topic: 5 Word: 0.016*"water" + 0.015*"govt" + 0.011*"plan" + 0.011*"council" + 0.010*"hick" + 0.010*"fund" + 0.010*"r

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [33]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [34]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4233505427837372	 
Topic: 0.025*"aust" + 0.021*"drought" + 0.019*"australia" + 0.017*"record" + 0.016*"break" + 0.015*"troop" + 0.015*"look" + 0.014*"rain" + 0.012*"dead" + 0.012*"timor"

Score: 0.2200000286102295	 
Topic: 0.025*"closer" + 0.021*"test" + 0.018*"lead" + 0.016*"take" + 0.014*"south" + 0.012*"law" + 0.012*"author" + 0.012*"inquiri" + 0.012*"crew" + 0.011*"bushfir"

Score: 0.21664941310882568	 
Topic: 0.054*"polic" + 0.024*"help" + 0.023*"miss" + 0.023*"seek" + 0.021*"urg" + 0.016*"search" + 0.012*"region" + 0.012*"arrest" + 0.011*"train" + 0.010*"offic"

Score: 0.02000001072883606	 
Topic: 0.060*"water" + 0.033*"warn" + 0.018*"worker" + 0.017*"rise" + 0.015*"industri" + 0.013*"rudd" + 0.012*"busi" + 0.012*"union" + 0.011*"begin" + 0.011*"continu"

Score: 0.020000005140900612	 
Topic: 0.070*"govt" + 0.039*"council" + 0.030*"plan" + 0.021*"fund" + 0.015*"urg" + 0.014*"defend" + 0.014*"back" + 0.013*"consid" + 0.013*"reject" + 0.012*"boost"

Score: 0.01999999769032

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [35]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199833035469055	 
Topic: 0.039*"closer" + 0.008*"govt" + 0.008*"damag" + 0.008*"water" + 0.007*"rain" + 0.007*"plant" + 0.007*"suppli" + 0.007*"increas" + 0.007*"worri" + 0.006*"propos"

Score: 0.020005416125059128	 
Topic: 0.011*"troop" + 0.011*"crew" + 0.010*"iraq" + 0.009*"qanta" + 0.009*"howard" + 0.009*"solomon" + 0.008*"takeov" + 0.008*"aust" + 0.007*"approv" + 0.007*"sailor"

Score: 0.020002814009785652	 
Topic: 0.013*"stab" + 0.012*"guilti" + 0.012*"drought" + 0.012*"murray" + 0.010*"black" + 0.008*"plead" + 0.008*"canberra" + 0.007*"grant" + 0.007*"charg" + 0.007*"thousand"

Score: 0.020002540200948715	 
Topic: 0.013*"coast" + 0.011*"south" + 0.010*"north" + 0.010*"west" + 0.009*"timor" + 0.009*"storm" + 0.009*"gold" + 0.009*"condit" + 0.008*"east" + 0.008*"iemma"

Score: 0.020002121105790138	 
Topic: 0.025*"crash" + 0.024*"polic" + 0.020*"investig" + 0.018*"kill" + 0.016*"miss" + 0.014*"search" + 0.010*"dead" + 0.010*"fatal" + 0.009*"bodi" + 0.009*"bomb"

Score: 0.

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [36]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4200001060962677	 Topic: 0.025*"power" + 0.018*"win" + 0.017*"nuclear" + 0.017*"hick" + 0.016*"minist"
Score: 0.2200000286102295	 Topic: 0.025*"closer" + 0.021*"test" + 0.018*"lead" + 0.016*"take" + 0.014*"south"
Score: 0.21999986469745636	 Topic: 0.060*"water" + 0.033*"warn" + 0.018*"worker" + 0.017*"rise" + 0.015*"industri"
Score: 0.020000001415610313	 Topic: 0.054*"polic" + 0.024*"help" + 0.023*"miss" + 0.023*"seek" + 0.021*"urg"
Score: 0.020000001415610313	 Topic: 0.025*"aust" + 0.021*"drought" + 0.019*"australia" + 0.017*"record" + 0.016*"break"
Score: 0.020000001415610313	 Topic: 0.019*"push" + 0.018*"elect" + 0.016*"labor" + 0.016*"howard" + 0.015*"high"
Score: 0.020000001415610313	 Topic: 0.033*"charg" + 0.031*"crash" + 0.029*"polic" + 0.029*"kill" + 0.029*"court"
Score: 0.020000001415610313	 Topic: 0.023*"group" + 0.018*"blaze" + 0.017*"return" + 0.015*"work" + 0.014*"question"
Score: 0.020000001415610313	 Topic: 0.028*"jail" + 0.024*"road" + 0.022*"year" + 0.021*"ope

The model correctly classifies the unseen document with 'x'% probability to the X category.